In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import matplotlib.pyplot as plt
from scipy import spatial
from sklearn.cluster import DBSCAN

!pip install changefinder
import changefinder

!pip install lasio
import lasio

In [3]:
def findChangePoints(ts, r, order, smooth):
    '''
       r: Discounting rate
       order: AR model order
       smooth: smoothing window size T
    '''
    cf = changefinder.ChangeFinder(r=r, order=order, smooth=smooth)
    ts_score = [cf.update(p) for p in ts]
    return(ts_score)


def get_change_points(var,r,order,smooth,n_largest):
    ts_score1 = findChangePoints(var, r = r, order = order, smooth = smooth)
    ts_change_loc1 = pd.Series(ts_score1).nlargest(n_largest)
    ts_change_loc1 = ts_change_loc1.index
    return ts_change_loc1

# **LAS_168**

In [4]:
df=lasio.read("../input/ovl-las-with-labels/1-BRSA-1088-SES-168_COMPOSITE.las")
well_name=df.well.WELL.value
df=df.df()
df=df.reset_index()
df=df[["DEPTH","GR","DTC","NPHI"]]
df=df.dropna()
df=df[df["GR"]>=0]
df=df[df["DTC"]>=0]
df=df[df["NPHI"]>=0]
df=df.reset_index(drop=True)
df

# GR

In [5]:
idx = get_change_points(var=df["GR"],r=0.1,order=1,smooth=5,n_largest=100).tolist()
    
depth_points = df.iloc[idx, :]
depth_points = list(depth_points['DEPTH'])
depth_points = sorted(depth_points)
depth_points

In [6]:
depth_keypoints1 = []
depth = list(df['DEPTH'])[0]
while depth <= list(df['DEPTH'])[-1]:
    x = []
    for val in depth_points:
        if val >= depth and val < depth + 10:
            x.append(val)
            
    depth += 10
    if len(x) > 0:
        y = sum(x)/len(x)
        depth_keypoints1.append(y)
    
len(depth_keypoints1)

In [7]:
fig, axes = plt.subplots(figsize=(12,48))

curve_names = ['GR', 'DTC', 'NPHI']

#Set up the plot axes
ax1 = plt.subplot2grid((1,1), (0,0), rowspan=1, colspan = 1)

#Set up the individual log tracks / subplots
ax1.plot("GR", "DEPTH", data = df, color = "red")

#Set up the common elements between the subplots
for i, ax in enumerate(fig.axes):
    ax.set_ylim(5500, 4100) # Set the depth range
    ax.set_xlabel(curve_names[0])
    for value in depth_keypoints1:
        ax.axhline(y=value, color='black', linestyle='-', linewidth=1)

plt.show()

# DTC

In [8]:
idx = get_change_points(var=df["DTC"],r=0.1,order=1,smooth=5,n_largest=100).tolist()
    
depth_points = df.iloc[idx, :]
depth_points = list(depth_points['DEPTH'])
depth_points = sorted(depth_points)
depth_points

In [9]:
depth_keypoints2 = []
depth = list(df['DEPTH'])[0]
while depth <= list(df['DEPTH'])[-1]:
    x = []
    for val in depth_points:
        if val >= depth and val < depth + 10:
            x.append(val)
            
    depth += 10
    if len(x) > 0:
        y = sum(x)/len(x)
        depth_keypoints2.append(y)
    
len(depth_keypoints2)

In [10]:
fig, axes = plt.subplots(figsize=(12,48))

curve_names = ['GR', 'DTC', 'NPHI']

#Set up the plot axes
ax1 = plt.subplot2grid((1,1), (0,0), rowspan=1, colspan = 1)

#Set up the individual log tracks / subplots
ax1.plot("DTC", "DEPTH", data = df, color = "blue")

#Set up the common elements between the subplots
for i, ax in enumerate(fig.axes):
    ax.set_ylim(5500, 4100) # Set the depth range
    ax.set_xlabel(curve_names[1])
    for value in depth_keypoints2:
        ax.axhline(y=value, color='black', linestyle='-', linewidth=1)

plt.show()

# NPHI

In [11]:
idx = get_change_points(var=df["NPHI"],r=0.1,order=1,smooth=5,n_largest=100).tolist()
    
depth_points = df.iloc[idx, :]
depth_points = list(depth_points['DEPTH'])
depth_points = sorted(depth_points)
depth_points

In [12]:
depth_keypoints3 = []
depth = list(df['DEPTH'])[0]
while depth <= list(df['DEPTH'])[-1]:
    x = []
    for val in depth_points:
        if val >= depth and val < depth + 10:
            x.append(val)
            
    depth += 10
    if len(x) > 0:
        y = sum(x)/len(x)
        depth_keypoints3.append(y)
    
len(depth_keypoints3)

In [13]:
fig, axes = plt.subplots(figsize=(12,48))

curve_names = ['GR', 'DTC', 'NPHI']

#Set up the plot axes
ax1 = plt.subplot2grid((1,1), (0,0), rowspan=1, colspan = 1)

#Set up the individual log tracks / subplots
ax1.plot("NPHI", "DEPTH", data = df, color = "green")

#Set up the common elements between the subplots
for i, ax in enumerate(fig.axes):
    ax.set_ylim(5500, 4100) # Set the depth range
    ax.set_xlabel(curve_names[2])
    for value in depth_keypoints3:
        ax.axhline(y=value, color='black', linestyle='-', linewidth=1)

plt.show()

In [14]:
print(set(depth_keypoints1) & set(depth_keypoints2) & set(depth_keypoints3))